In [2]:
import numpy as np
import pandas as pd
import urllib
import json
from datetime import date
from datetime import timedelta

with open(".key", "r") as f:
    key = f.read()
    
channelIds = {'bbcUzbek': 'UCGu3z18dm6hqQAScxG12qBg',
    'bbcFarsi': 'UCHZk9MrT3DGWmVqdsj5y0EA'}

In [3]:
searchResults = {}
videoResults = {}

In [12]:
def getLatestVideosForChannel(name):
    if name in searchResults.keys():
        searchResult = searchResults[name]
    else:
        url = ('https://www.googleapis.com/youtube/v3/search?channelId={}&key={}&order=date'.format(channelIds[name], key)
          + '&maxResults=50')

        with urllib.request.urlopen(url) as f:
            searchResult = f.read().decode('utf-8')
            searchResults[name] = searchResult
        
    videoIds = ','.join(item['id']['videoId'] 
                    for item in json.loads(searchResult)['items'] if item['id']['kind'] == 'youtube#video')
    
    if name in videoResults.keys():
        videoResult = videoResults[name]
    else:
        url = ('https://www.googleapis.com/youtube/v3/videos?id={}&key={}&maxResults=50'.format(videoIds, key)
              + '&part=snippet,contentDetails')

        with urllib.request.urlopen(url) as f:
            videoResult = f.read().decode('utf-8')
            videoResults[name] = videoResult
    
    videoJson = json.loads(videoResult)

    itemRows = []
    for item in videoJson['items']:
        itemRow = {'id': item['id']}
        for snippetKey in item['snippet'].keys():
            itemRow[snippetKey] = str(item['snippet'][snippetKey])
        itemRow['duration'] = item['contentDetails']['duration']
        itemRow['licensedContent'] = item['contentDetails']['licensedContent']
        itemRows.append(itemRow)

    itemDf = pd.DataFrame(itemRows)[['id', 'publishedAt', 'title', 'duration']].sort_values('publishedAt', ascending=False)
    
    today = date.today()
    todayStr = today.strftime('%Y-%m-%d')
    yesterdayStr = (today - timedelta(days = 1)).strftime('%Y-%m-%d')
    itemDf = itemDf[(itemDf['publishedAt'].str.startswith(todayStr))|(itemDf['publishedAt'].str.startswith(yesterdayStr))]
    
    itemDf['minutes'] = itemDf.duration.apply(lambda d: isodate.parse_duration(d).total_seconds() / 60.)
    itemDf = itemDf[itemDf['minutes'] >= 10]

    return itemDf

In [15]:
getLatestVideosForChannel('bbcFarsi')

,id,publishedAt,title,duration,minutes
0,K-hAifbChmE,2021-10-14T21:51:04Z,در شرایط بد اقتصادی چرا مجلس به اجرای طرح صیان...,PT54M22S,54.366667
2,8o5f3i9HEOM,2021-10-14T15:26:57Z,اخبار ساعت شش عصر- پنج‌شنبه ۲۲ مهر,PT28M21S,28.350000
3,U7hIlCUX9cw,2021-10-13T23:08:50Z,هفت تپه خصوصی یا دولتی، کدام به نفع کارگران اس...,PT57M15S,57.250000
4,Fn32SFa6Vq0,2021-10-13T16:00:08Z,اولین قرص خوراکی موثر کرونا وپاسخ به سوالات شم...,PT23M28S,23.466667
5,pKHeNruy2lA,2021-10-13T15:27:42Z,اخبار ساعت شش عصر- چهارشنبه ۲۱ مهر,PT27M46S,27.766667
